#Yelp review data#

In [88]:
import json
import urllib2
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob, Word
from pandas.io.json import json_normalize
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer

%matplotlib inline

In [20]:
#url = 'data/yelp.json'
url = 'file:///Users/ziru/DS/Profile/data/yelp.json'
data = urllib2.urlopen(url)
records = [json.loads(line) for line in data]

In [22]:
records[:1]

[{u'business_id': u'9yKzy9PApeiPPOUJEtnvkg',
  u'date': u'2011-01-26',
  u'review_id': u'fWKvX83p0-ka4JS3dc6E5A',
  u'stars': 5,
  u'text': u'My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever 

In [25]:
# json_normalize() will also take care of embedded json strings, see votes.cool for example
records_frame = json_normalize(records)

In [26]:
records_frame[:3]

,business_id,date,review_id,stars,text,type,user_id,votes.cool,votes.funny,votes.useful
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,0,5
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,0,1


In [27]:
records_frame.rename(columns={'votes.cool':'cool','votes.useful':'useful','votes.funny':'funny'}, inplace=True)

**See if the same user gave more than one reviews...turns out the top user has 38 reviews in this sample**

In [28]:
review_group = records_frame.groupby(['user_id']).size()

In [29]:
review_group = pd.DataFrame(review_group)

In [30]:
review_group.columns=['review_counts']

In [32]:
review_group.sort_values(by='review_counts',ascending=False)[:20]

,review_counts
user_id,
fczQCSmaWF78toLEmb0Zsw,38
0CMz8YaO3f8xu4KqQgKb9Q,25
90a6z--_CUrl84aCzZyPsg,22
4ozupHULqGyO42s3zNUzOQ,18
0mqHhdKEdak_A1FBhFNXqA,18
wHg1YkCzdZq9WBJOTRgxHQ,17
0bNXP9quoJEgyVZu9ipGgQ,16
_PzSNcfrCjeBxSLXRoMmgQ,16
JgDkCER12uiv4lbpmkZ9VA,15


In [33]:
yelp = records_frame[:]

In [38]:
# Get records with either 5 stars or 1 stars
yelp_best_worst = yelp[(yelp.stars==5)|(yelp.stars==1)]

In [37]:
yelp_best_worst[:5]

,business_id,date,review_id,stars,text,type,user_id,cool,funny,useful
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,0,5
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,0,2
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
6,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,4,7


**Using Yelp users' review to predict the stars users are giving**

In [55]:
X = yelp_best_worst.text

In [56]:
y = yelp_best_worst.stars

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [82]:
# This can test different vectorizers
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print 'Features: ', X_train_dtm.shape[1]
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    print 'Accuracy: ', metrics.accuracy_score(y_test, y_pred_class)

In [83]:
# Tokenize the text
#vect = CountVectorizer()
vect = CountVectorizer(lowercase=False)
tokenize_test(vect)

Features:  20838
Accuracy:  0.909980430528


In [84]:
vect = CountVectorizer()
tokenize_test(vect)

Features:  16825
Accuracy:  0.918786692759


In [85]:
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  169847
Accuracy:  0.854207436399


**Remove stopwords**

In [86]:
# remove English stop words
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  16528
Accuracy:  0.915851272016


In [87]:
print vect.get_stop_words()

frozenset(['all', 'six', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'four', 'not', 'own', 'through', 'yourselves', 'fify', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'with', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'under', 'ours', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'whether', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'several', 'hereafter', 'always', 'who', 'cry', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'two', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'eg', 'some', 'back', 'up', 'go', 'namely', 'towards', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its', 'everything', 'behind', 'un', 'above', 'between', 'it', 'neit

**Stemming and Lemmatization**

In [92]:
# initialize stemmer
stemmer = SnowballStemmer('english')

# Here TextBlob is needed to get the text
def split_into_lemmas(text):
    text = unicode(text).lower()
    words = TextBlob(text).words
    return [word.lemmatize() for word in words]

In [93]:
vect = CountVectorizer(analyzer=split_into_lemmas)
tokenize_test(vect)

Features:  16452
Accuracy:  0.920743639922
